In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  # workaround for buggy jedi

In [ ]:
# default_exp validation

In [ ]:
#export
from sql_formatter.utils import *

# validation

> Functions for basic SQL validation

In [ ]:
#hide
from nbdev.showdoc import *

## Motivation

Sometimes, users may make some syntax errors that in turn makes the formatter fail. We want therefore to catch this errors before formatting and give the user a hint to where the error lies

### Missing semicolon to separate queries

One mistake that would make the formatter fail is if the SQL queries are not properly delimited by semicolon. We therefore make a basic validation and look for the keyword CREATE appearing twice in a query. As this cannot happen within one query, the validation should fail and point the user out that she / he may have forgotten a semicolon

In [ ]:
#export
def validate_semicolon(s):
    """Validate query `s` by looking for forgotten semicolon. 
    The implication could be the keyword CREATE appearing twice"""
    positions = identify_create(s)
    if len(positions) > 1:
        validation = {
            "exit_code": 1,
            "val_lines": find_line_number(s, positions),
            "total_lines": count_lines(s)
        }
    else:
        validation = {
            "exit_code": 0,
            "total_lines": count_lines(s)
        }
    return validation

In [ ]:
assert_and_print(
    validate_semicolon(
"""
create or replace table my_table as
select asdf, qwer from table1

create view my_view as select asdf from my_table
"""
    ), {"exit_code": 1, "val_lines": [2, 5], "total_lines": 5}
)

{'exit_code': 1, 'val_lines': [2, 5], 'total_lines': 5}


In [ ]:
assert_and_print(
    validate_semicolon(
"""
create or replace table my_table as
select asdf, qwer from table1

"""
    ), {"exit_code": 0, "total_lines": 4}
)

{'exit_code': 0, 'total_lines': 4}


### Unbalanced parenthesis

In [ ]:
#export
def validate_balanced_parenthesis(s):
    """Validate query `s` by looking for
    unbalanced parenthesis
    
    exit_code:
    * 0 = balanced parenthesis
    * 1 = unbalanced parenthesis, too many (
    * 2 = unbalanced parenthesis, too many )
    """
    positions = []  # container for position of unbalanced parenthesis
    # counter for comments
    k = 0  # 0 = no comment range
    comment_open1 = False # comment indicator for /* */ comments
    comment_open2 = False  # comment indicator for -- comments
    quote_open1 = False  # quote '
    quote_open2 = False # quote "
    for i, c in enumerate(s):
        if c == "(" and k == 0:
            positions.append(i)
        elif c == ")" and k == 0:
            if len(positions) == 0:
                return {
                    "exit_code": 1,
                    "val_lines": find_line_number(s, [i]),
                    "total_lines": count_lines(s)
                }
            else:
                positions.pop()
        elif (
            s[i:i+2] == "/*" and 
            not comment_open1 and 
            not comment_open2 and
            not quote_open1 and 
            not quote_open2 
        ):  # if there is an opening comment /*
            k += 1
            comment_open1 = True
        elif (
            s[i:i+2] == "*/" and
            comment_open1 and
            not comment_open2 and
            not quote_open1 and
            not quote_open2
        ):  # if there is a closing comment */
            k -= 1
            comment_open1 = False
        elif (
            s[i:i+2] == "--" and 
            not comment_open1 and 
            not comment_open2 and
            not quote_open1 and 
            not quote_open2 
        ):  # if there is an opening comment --
            k += 1
            comment_open2 = True
        elif (
            (c == "\n" or s[i:i+3] == "[c]") and
            not comment_open1 and
            comment_open2 and
            not quote_open1 and
            not quote_open2
        ):  # if the -- comment ends
            k -= 1
            comment_open2 = False
        elif (
            c == "'" and
            not comment_open1 and 
            not comment_open2 and
            not quote_open1 and 
            not quote_open2            
        ):  # if opening quote '
            k += 1
            quote_open1 = True
        elif (
            c == "'" and
            not comment_open1 and 
            not comment_open2 and
            quote_open1 and 
            not quote_open2            
        ):  # if opening quote '
            k -= 1
            quote_open1 = False
        elif (
            c == '"' and
            not comment_open1 and 
            not comment_open2 and
            not quote_open1 and 
            quote_open2            
        ):  # if opening quote '
            k += 1
            quote_open2 = True
        elif (
            c == '"' and
            not comment_open1 and 
            not comment_open2 and
            not quote_open1 and 
            quote_open2            
        ):  # if opening quote '
            k -= 1
            quote_open2 = False
    if len(positions) == 0:
        return {
            "exit_code": 0,
            "total_lines": count_lines(s)
        }
    else:
        return {
            "exit_code": 1,
            "val_lines": find_line_number(s, positions),                
            "total_lines": count_lines(s)
        }

In [ ]:
    validate_balanced_parenthesis("() () ( () )")

{'exit_code': 0, 'total_lines': 0}

In [ ]:
assert_and_print(
    validate_balanced_parenthesis("() () ( () )"),
    {"exit_code": 0, "total_lines": 0}
)

{'exit_code': 0, 'total_lines': 0}


In [ ]:
assert_and_print(
    validate_balanced_parenthesis(
"""
(
(
)
(
"""
    ),
    {"exit_code": 1, "val_lines": [2, 5], "total_lines": 5}
)

{'exit_code': 1, 'val_lines': [2, 5], 'total_lines': 5}


In [ ]:
assert_and_print(
    validate_balanced_parenthesis(
"""
( )
-- ) ( )( ) ()
)
"""
    ),
    {"exit_code": 1, "val_lines": [4], "total_lines": 4}
)

{'exit_code': 1, 'val_lines': [4], 'total_lines': 4}


In [ ]:
assert_and_print(
    validate_balanced_parenthesis("( )( )"),
    {"exit_code": 0, "total_lines": 0}
)

{'exit_code': 0, 'total_lines': 0}


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_format_file.ipynb.
Converted 02_utils.ipynb.
Converted 03_validation.ipynb.
Converted index.ipynb.
